# L4a: Kernel Functions and Kernel Regression
Fill me in

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [3]:
include("Include.jl");

### Data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2014` until `02-07-2025`, along with data for a few exchange-traded funds and volatility products during that time. We load the `orignal_dataset` by calling the `MyMarketDataSet()` function:

In [5]:
original_dataset = MyMarketDataSet() |> x-> x["dataset"];


__Clean the data__: Not all tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquisition or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days` variable:

In [7]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [9]:
dataset = let

    dataset = Dict{String,DataFrame}();
    for (ticker,data) ∈ original_dataset
        if (nrow(data) == maximum_number_trading_days)
            dataset[ticker] = data;
        end
    end
    dataset
end;

Let's get a list of firms in the cleaned up `dataset` and save it in the `all_tickers` array. We sort the firms alphabetically from `A` to `Z`:

In [11]:
list_of_all_tickers = keys(dataset) |> collect |> sort;

Compute the expected (annualized) excess log growth rate by passing the `dataset` and the entire list of firms we have in the dataset to the [log_growth_matrix(...) method](src/Compute.jl). The log growth rate between time period $j-1$ to $j$, e.g., yesterday to today is defined as:
$$
\begin{equation}
\mu_{j,j-1} = \left(\frac{1}{\Delta{t}}\right)\ln\left(\frac{S_{j}}{S_{j-1}}\right)
\end{equation}
$$
where $\Delta{t}$ denotes the period time step, and $S_{j}$ denote share price in period $j$.
* The log growth rates are stored in the `D::Array{Float64,2}` variable, a $T-1\times{N}$ array of log return values. Each row of the `D` matrix corresponds to a time value, while each column corresponds to a firm

In [27]:
D = let

    # setup some constants -
    Δt = (1/252); # 1-trading day in units of years
    risk_free_rate = 0.0415; # inferred cc risk-free rate

    # compute
    μ = log_growth_matrix(dataset, list_of_all_tickers, Δt = Δt, 
        risk_free_rate = risk_free_rate);

    # return to caller
    μ
end

2791×424 Matrix{Float64}:
 -0.919054     6.23955    -2.91247    …   -0.796891    0.204394  -1.04677
  2.77476      1.02999     1.35089         2.09682    -0.84429    0.944968
  3.27155      0.814097   -0.036132        0.068377    1.1495    -2.62294
  0.604925    17.2184      1.65065         0.233216    3.1178    -0.409728
  1.77459      2.53811     3.27774         0.580177   -2.2102     4.36159
  0.57233     -4.00534    -0.83428    …   -0.904239   -1.95127   -3.15774
  2.81921     -0.525251    4.80423         1.7242     -1.81835   -1.1311
  2.00521      0.972004    1.86659         1.63447     4.40834   -0.179319
  1.27139      1.63263     0.0657592      -1.54858    -2.17846    1.39634
  1.17866      6.08807     0.891078       -1.57352     2.83634   -1.47776
 -0.479168     4.82859     0.96624    …   -0.362761    9.46677   -3.05023
  1.32131      3.57167    -2.38926         0.669113    4.48073    0.299031
 -4.78054      1.34435    -3.05774        -2.19395    -6.69057    1.36462
  ⋮      

Next, let's [z-score center](https://en.wikipedia.org/wiki/Feature_scaling) the continous feature data. In [z-score feature scaling](https://en.wikipedia.org/wiki/Feature_scaling), we subtract off the mean of each feature and then divide by the standard deviation, i.e., $x^{\prime} = (x - \mu)/\sigma$ where $x$ is the unscaled data, and $x^{\prime}$ is the scaled data. Under this scaling regime, $x^{\prime}\leq{0}$ will be values that are less than or equal to the mean value $\mu$, while $x^{\prime}>0$ indicate values that are greater than the mean.

We save the z-score centered growth data data in the `D̄::Array{Float64,2}` variable:

In [32]:
D̄ = let

    # setup -
    number_of_examples = size(D,1);

    D̄ = copy(D);
    for j ∈ eachindex(list_of_all_tickers)
        μ = mean(D[:,j]); # compute the mean
        σ = std(D[:,j]); # compute std

        # rescale -
        for k ∈ 1:number_of_examples
            D̄[k,j] = (D[k,j] - μ)/σ;
        end
    end
    
    D̄
end

2791×424 Matrix{Float64}:
 -0.270236    0.875241   -0.51285    …  -0.172603    0.0340086  -0.326599
  0.735235    0.153683    0.270667       0.365351   -0.156963    0.235764
  0.870466    0.123779    0.0157621     -0.0117454   0.206118   -0.771629
  0.144598    2.39589     0.325758       0.018899    0.564557   -0.146732
  0.462985    0.362567    0.624783       0.0834006  -0.405704    1.20044
  0.135725   -0.543745   -0.130921   …  -0.192559   -0.35855    -0.922629
  0.747335   -0.0617295   0.905321       0.29608    -0.334345   -0.35041
  0.52576     0.145651    0.365443       0.279399    0.799572   -0.0816769
  0.326013    0.237151    0.0344876     -0.312346   -0.399924    0.363208
  0.30077     0.854261    0.186164      -0.316981    0.513302   -0.448288
 -0.150497    0.679814    0.199977   …  -0.0918959   1.72074    -0.892273
  0.339601    0.505721   -0.416694       0.0999343   0.812754    0.0533844
 -1.32135     0.197223   -0.539546      -0.432322   -1.22161     0.354251
  ⋮         

## Kernel Functions
Kernel functions in machine learning are mathematical tools that enable algorithms to operate in high-dimensional spaces without explicitly computing the coordinates in those spaces.

* __What are kernel functions__? A kernel function $k:\mathbb{R}^{\star}\times\mathbb{R}^{\star}\to\mathbb{R}$ takes a pair of vectors $\mathbf{z}_i\in\mathbb{R}^{\star}$ and $\mathbf{z}_j\in\mathbb{R}^{\star}$ as arguments, 
e.g., a pair of feature vectors, a feature vector and a parameter vector, or any two vectors of compatible size 
, computes a scalar value that represents the similarity (in some sense) between the two vector arguments.
* __Common kernel functions__: Common kernel functions include the linear kernel, which is the dot product between two vectors $k(\mathbf{z}_i, \mathbf{z}_j) = \mathbf{z}_i^{\top}\mathbf{z}_j$, the polynomial kernel is defined as: $k_{d}(\mathbf{z}_i, \mathbf{z}_j) = (1+\mathbf{z}_i^{\top}\mathbf{z}_j)^d$ and the radial basis function (RBF) kernel $k_{\gamma}(\mathbf{z}_i, \mathbf{z}_j) = \exp(-\gamma \lVert\mathbf{z}_i - \mathbf{z}_j\rVert_{2}^2)$ where $\gamma$ is a scaling factor, and $\lVert\cdot\rVert^{2}_{2}$ is the squared Euclidean norm
* __What do kernel functions do__? Kernel functions can be thought of as similarity measures, i.e., they quantify the similarity between pairs of data points in a high-dimensional space.